In [11]:
import numpy as np
import tensorflow as tf

In [ ]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")

In [4]:
training = tf.placeholder_with_default(False, shape = (), name = "training")

In [5]:
hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training = training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name = "hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training = training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name = "outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training = training, momentum=0.9)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.


# コードの重複をパーティシャルを使ってわかりやすくする

In [5]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [6]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [111]:
from functools import partial

In [159]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int32, shape = (None), name = "y")
training = tf.placeholder_with_default(False, shape = (), name = "training")
my_batch_norm_layer = partial(tf.layers.batch_normalization, training = training, momentum = 0.9)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1")
    bn1 = my_batch_norm_layer(hidden1)
    bn1_act = tf.nn.elu(bn1)
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name = "hidden2")
    bn2 = my_batch_norm_layer(hidden2)
    bn2_act = tf.nn.elu(bn2)
    logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name = "outputs")
    logits = my_batch_norm_layer(logits_before_bn)
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = "loss")
    loss_summary = tf.summary.scalar('log_loss', loss)
with tf.name_scope("train"):
    learning_rate = 0.01
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
with tf.name_scope("init"):
    init = tf.global_variables_initializer()
with tf.name_scope("saver"):
    saver = tf.train.Saver()

In [160]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [161]:
n_epochs = 101
batch_size = 200

In [162]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops], feed_dict={training:True, X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 20 == 0:
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 0.87 Validation accuracy: 0.8824
20 Batch accuracy: 0.99 Validation accuracy: 0.9686
40 Batch accuracy: 0.995 Validation accuracy: 0.9756
60 Batch accuracy: 0.99 Validation accuracy: 0.9774
80 Batch accuracy: 1.0 Validation accuracy: 0.9784
100 Batch accuracy: 1.0 Validation accuracy: 0.9788


# 勾配クリッピング

In [157]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int32, shape = (None), name = "y")
training = tf.placeholder_with_default(False, shape = (), name = "training")
my_batch_norm_layer = partial(tf.layers.batch_normalization, training = training, momentum = 0.9)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1")
    bn1 = my_batch_norm_layer(hidden1)
    bn1_act = tf.nn.elu(bn1)
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name = "hidden2")
    bn2 = my_batch_norm_layer(hidden2)
    bn2_act = tf.nn.elu(bn2)
    logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name = "outputs")
    logits = my_batch_norm_layer(logits_before_bn)
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = "loss")
    loss_summary = tf.summary.scalar('log_loss', loss)
with tf.name_scope("train"):
    learning_rate = 0.01
    threshold = 1.0
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads_and_vars = optimizer.compute_gradients(loss)#勾配を計算
    capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]#クリッピング
    training_op = optimizer.apply_gradients(capped_gvs)#クリッピング適応
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
with tf.name_scope("init"):
    init = tf.global_variables_initializer()
with tf.name_scope("saver"):
    saver = tf.train.Saver()

In [158]:
n_epochs = 401
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 50 == 0:
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_normalization_model_final.ckpt")

0 Batch accuracy: 0.86 Validation accuracy: 0.8594
50 Batch accuracy: 0.96 Validation accuracy: 0.962
100 Batch accuracy: 0.995 Validation accuracy: 0.974
150 Batch accuracy: 0.995 Validation accuracy: 0.9764
200 Batch accuracy: 1.0 Validation accuracy: 0.9794
250 Batch accuracy: 1.0 Validation accuracy: 0.9794
300 Batch accuracy: 1.0 Validation accuracy: 0.98
350 Batch accuracy: 1.0 Validation accuracy: 0.9802
400 Batch accuracy: 1.0 Validation accuracy: 0.9806
450 Batch accuracy: 1.0 Validation accuracy: 0.9806


KeyboardInterrupt: 

# プレトレーニング済み層の再利用

## TensorFlowモデルの再利用

In [163]:
reset_graph()

In [164]:
saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")#計算グラフを取ってきて、それをデフォルトグラフにする

In [165]:
#オペレーションの確認
for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
training/input
training
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
batch_normalization/gamma/Initializer/ones
batch_normalization/gamma
batch_normalization/gamma/Assign
batch_normalization/gamma/read
batch_normalization/beta/Initializer/zeros
batch_normalization/beta
batch_normalization/beta/Assign
batch_normalization/beta/read
batch_normalization/moving_mean/Initializer/zeros
batch_normalization/moving_mean
batch_normalization/moving_mean/Assign
batch_normalization/moving_mean/read
batch_normalization/moving_variance/Initial

## 保存した計算グラフのテンソル計算などを取得

In [166]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
training_op = tf.get_default_graph().get_operation_by_name("train/GradientDescent")

## オペレーションをコレクションで再利用しやすくする

In [167]:
for op in (X, y, accuracy, training_op):
    tf.add_to_collection("my_important_ops", op)

In [168]:
#コレクションを使用
X, y, accuracy, training_op = tf.get_collection("my_important_ops")

In [169]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)
    save_path = saver.save(sess, "./my_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
1 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
2 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
3 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
4 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
5 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
6 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
7 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
8 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
9 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
10 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
11 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
12 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
13 Validation accuracy: b'\n\x14\n\reval/accuracy\x15\xc8\x98{?'
14 Validation accuracy: b'\n\x14\n\r

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

In [4]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
training_op = tf.get_default_graph().get_operation_by_name("train/GradientDescent")

## 層の途中までを違うデータセットで使う場合はこのように定義する必要がある

In [22]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01
threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [24]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key hidden3/bias not found in checkpoint
	 [[node save/RestoreV2 (defined at <ipython-input-22-60b577237cae>:40) ]]

Caused by op 'save/RestoreV2', defined at:
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.

In [21]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")       # reused
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # reused
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # reused
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):                                            # not shown in the book
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): # not shown
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})        # not shown
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})     # not shown
        print(epoch, "Validation accuracy:", accuracy_val)                   # not shown

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [300,50] rhs shape= [300,100]
	 [[node save/Assign_3 (defined at <ipython-input-19-22525adb8620>:1) ]]

Caused by op 'save/Assign_3', defined at:
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
 

## 下位層の凍結

In [ ]:
#3,4，出力層のみを渡すやり方
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = "hidden[34]|outputs")
training_op = optimizer.minimize(loss, var_list = train_vars)

In [ ]:
#凍結する層をしてするやり方
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name = "hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name = "hidden2")
    hidden2_stop = tf.stop_gradient(hidden2)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name = "hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name = "hidden4")
    logits = tf.layers.dense(hidden4, n_outputs, name = "outputs")

# 凍結層のキャッシング

In [25]:
import numpy as np

In [27]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              name="hidden1") # reused frozen
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              name="hidden2") # reused frozen & cached
    hidden2_stop = tf.stop_gradient(hidden2)
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu,
                              name="hidden3") # reused, not frozen
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu,
                              name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs") # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

import numpy as np

batch_size = 20
n_batches = len(X_train) // batch_size

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")
    
    h2_cache = sess.run(hidden2, feed_dict={X: X_train})
    h2_cache_valid = sess.run(hidden2, feed_dict={X: X_valid}) # not shown in the book

    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(len(X_train))
        hidden2_batches = np.array_split(h2_cache[shuffled_idx], n_batches)
        y_batches = np.array_split(y_train[shuffled_idx], n_batches)
        for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
            sess.run(training_op, feed_dict={hidden2:hidden2_batch, y:y_batch})

        accuracy_val = accuracy.eval(feed_dict={hidden2: h2_cache_valid, # not shown
                                                y: y_valid})             # not shown
        print(epoch, "Validation accuracy:", accuracy_val)               # not shown

    save_path = saver.save(sess, "./my_new_model_final.ckpt")


INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key hidden3/bias not found in checkpoint
	 [[node save/RestoreV2 (defined at <ipython-input-27-017c9eb1f6d3>:39) ]]

Caused by op 'save/RestoreV2', defined at:
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hiroshi/anaconda3/envs/py36/lib/python3.

# 演習問題

In [34]:
import numpy as np
import tensorflow as tf
from datetime import datetime
import os

In [35]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [42]:
def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs

In [48]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

# 早期打ち切り

In [49]:
#構築フェーズ
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

n_inputs = 28*28
n_outputs = 5
n_hidden1 = 300
n_hidden2 = 100

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int32, shape = (None), name = "y")

with tf.name_scope("dnn"):
    he_init = tf.contrib.layers.variance_scaling_initializer()
    hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1", activation = tf.nn.elu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name = "hidden2", activation = tf.nn.elu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden2, n_outputs, name = "outputs")
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = "loss")
    loss_summary = tf.summary.scalar('log_loss', loss)
with tf.name_scope("train"):
    learning_rate = 0.01
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
with tf.name_scope("init"):
    init = tf.global_variables_initializer()
with tf.name_scope("saver"):
    saver = tf.train.Saver()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [50]:
train_under5_index = y_train < 5
valid_under5_index = y_valid < 5
test_under5_index = y_test < 5
under5_X_train, under5_y_train = X_train[train_under5_index], y_train[train_under5_index]
under5_X_valid, under5_y_valid = X_valid[valid_under5_index], y_valid[valid_under5_index]
under5_X_test, under5_y_test = X_test[test_under5_index], y_test[test_under5_index]

In [51]:
under5_X_train.shape

(28038, 784)

In [55]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(under5_X_train))
        for rnd_indices in np.array_split(rnd_idx, len(under5_X_train) // batch_size):
            X_batch, y_batch = under5_X_train[rnd_indices], under5_y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: under5_X_valid, y: under5_y_valid})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: under5_X_test, y: under5_y_test})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.096388	Best loss: 0.096388	Accuracy: 97.34%
1	Validation loss: 0.078388	Best loss: 0.078388	Accuracy: 98.36%
2	Validation loss: 0.074061	Best loss: 0.074061	Accuracy: 98.51%
3	Validation loss: 0.102700	Best loss: 0.074061	Accuracy: 97.38%
4	Validation loss: 0.145665	Best loss: 0.074061	Accuracy: 97.34%
5	Validation loss: 0.095697	Best loss: 0.074061	Accuracy: 98.71%
6	Validation loss: 0.191923	Best loss: 0.074061	Accuracy: 96.60%
7	Validation loss: 0.092960	Best loss: 0.074061	Accuracy: 98.36%
8	Validation loss: 0.142077	Best loss: 0.074061	Accuracy: 98.20%
9	Validation loss: 0.125678	Best loss: 0.074061	Accuracy: 98.71%
10	Validation loss: 0.217029	Best loss: 0.074061	Accuracy: 97.34%
11	Validation loss: 0.163078	Best loss: 0.074061	Accuracy: 98.91%
12	Validation loss: 0.420874	Best loss: 0.074061	Accuracy: 98.55%
13	Validation loss: 1.021748	Best loss: 0.074061	Accuracy: 97.89%
14	Validation loss: 0.124872	Best loss: 0.074061	Accuracy: 98.87%
15	Validation loss: 

In [56]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [57]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(under5_X_train, under5_y_train, n_epochs=1000, X_valid=under5_X_valid, y_valid=under5_y_valid)

0	Validation loss: 8.661100	Best loss: 8.661100	Accuracy: 41.09%
1	Validation loss: 0.328042	Best loss: 0.328042	Accuracy: 93.55%
2	Validation loss: 1.186421	Best loss: 0.328042	Accuracy: 61.57%
3	Validation loss: 0.497491	Best loss: 0.328042	Accuracy: 70.72%
4	Validation loss: 1.129167	Best loss: 0.328042	Accuracy: 95.93%
5	Validation loss: 0.199199	Best loss: 0.199199	Accuracy: 96.95%
6	Validation loss: 0.365291	Best loss: 0.199199	Accuracy: 94.18%
7	Validation loss: 0.300537	Best loss: 0.199199	Accuracy: 93.78%
8	Validation loss: 0.301715	Best loss: 0.199199	Accuracy: 97.26%
9	Validation loss: 0.328348	Best loss: 0.199199	Accuracy: 97.19%
10	Validation loss: 1.011822	Best loss: 0.199199	Accuracy: 97.34%
11	Validation loss: 78.367470	Best loss: 0.199199	Accuracy: 96.95%
12	Validation loss: 0.426976	Best loss: 0.199199	Accuracy: 85.50%
13	Validation loss: 0.268600	Best loss: 0.199199	Accuracy: 97.07%
14	Validation loss: 0.570963	Best loss: 0.199199	Accuracy: 95.70%
15	Validation loss:

DNNClassifier(activation=<function elu at 0x7f5dc2d7e268>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x7f5e3f715598>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)]
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                cv=3, random_state=42, verbose=2)
rnd_search.fit(under5_X_train, under5_y_train, X_valid=under5_X_valid, y_valid=under5_y_valid, n_epochs=1000)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=10, learning_rate=0.05, batch_size=100, activation=<function elu at 0x7f5dc2d7e268> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.130791	Best loss: 0.130791	Accuracy: 96.56%
1	Validation loss: 0.139919	Best loss: 0.130791	Accuracy: 96.44%
2	Validation loss: 0.181549	Best loss: 0.130791	Accuracy: 95.93%
3	Validation loss: 0.126994	Best loss: 0.126994	Accuracy: 96.52%
4	Validation loss: 1.201436	Best loss: 0.126994	Accuracy: 41.40%
5	Validation loss: 1.164190	Best loss: 0.126994	Accuracy: 39.52%
6	Validation loss: 1.319288	Best loss: 0.126994	Accuracy: 39.41%
7	Validation loss: 1.163483	Best loss: 0.126994	Accuracy: 42.26%
8	Validation loss: 1.161178	Best loss: 0.126994	Accuracy: 39.68%
9	Validation loss: 1.160900	Best loss: 0.126994	Accuracy: 42.57%
10	Validation loss: 1.225387	Best loss: 0.126994	Accuracy: 41.79%
11	Validation loss: 1.204676	Best loss: 0.126994	Accuracy: 42.53%
12	Validation loss: 1.171459	Best loss: 0.126994	Accuracy: 42.46%
13	Validation loss: 1.394094	Best loss: 0.126994	Accuracy: 38.90%
14	Validation loss: 1.176004	Best loss: 0.126994	Accuracy: 38.94%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.7s remaining:    0.0s


0	Validation loss: 0.223849	Best loss: 0.223849	Accuracy: 94.33%
1	Validation loss: 0.112426	Best loss: 0.112426	Accuracy: 96.72%
2	Validation loss: 0.122914	Best loss: 0.112426	Accuracy: 97.11%
3	Validation loss: 0.130797	Best loss: 0.112426	Accuracy: 96.95%
4	Validation loss: 0.123406	Best loss: 0.112426	Accuracy: 96.76%
5	Validation loss: 0.113320	Best loss: 0.112426	Accuracy: 96.91%
6	Validation loss: 0.171739	Best loss: 0.112426	Accuracy: 95.66%
7	Validation loss: 0.127681	Best loss: 0.112426	Accuracy: 96.76%
8	Validation loss: 0.138118	Best loss: 0.112426	Accuracy: 96.83%
9	Validation loss: 1.390939	Best loss: 0.112426	Accuracy: 36.12%
10	Validation loss: 1.559019	Best loss: 0.112426	Accuracy: 37.57%
11	Validation loss: 1.196293	Best loss: 0.112426	Accuracy: 41.20%
12	Validation loss: 1.244903	Best loss: 0.112426	Accuracy: 38.27%
13	Validation loss: 1.175448	Best loss: 0.112426	Accuracy: 38.39%
14	Validation loss: 1.124233	Best loss: 0.112426	Accuracy: 45.97%
15	Validation loss: 

...

7	Validation loss: 8.620263	Best loss: 8.620263	Accuracy: 92.34%
8	Validation loss: 7.349438	Best loss: 7.349438	Accuracy: 92.53%
9	Validation loss: 7.801530	Best loss: 7.349438	Accuracy: 93.39%
10	Validation loss: 9.928519	Best loss: 7.349438	Accuracy: 88.15%
11	Validation loss: 6.156703	Best loss: 6.156703	Accuracy: 92.92%
12	Validation loss: 5.725346	Best loss: 5.725346	Accuracy: 93.43%
13	Validation loss: 10.737847	Best loss: 5.725346	Accuracy: 81.63%
14	Validation loss: 8.583661	Best loss: 5.725346	Accuracy: 92.89%
15	Validation loss: 7.446820	Best loss: 5.725346	Accuracy: 90.89%
16	Validation loss: 4.820675	Best loss: 4.820675	Accuracy: 95.11%
17	Validation loss: 5.434150	Best loss: 4.820675	Accuracy: 94.45%
18	Validation loss: 4.432603	Best loss: 4.432603	Accuracy: 94.92%
19	Validation loss: 4.507710	Best loss: 4.432603	Accuracy: 95.58%
20	Validation loss: 3.812494	Best loss: 3.812494	Accuracy: 95.27%
21	Validation loss: 4.212516	Best loss: 3.812494	Accuracy: 92.22%
22	Validatio

28	Validation loss: 0.073438	Best loss: 0.041903	Accuracy: 98.59%
29	Validation loss: 0.063685	Best loss: 0.041903	Accuracy: 98.75%
30	Validation loss: 0.079315	Best loss: 0.041903	Accuracy: 98.87%
31	Validation loss: 0.102748	Best loss: 0.041903	Accuracy: 98.63%
32	Validation loss: 0.079225	Best loss: 0.041903	Accuracy: 98.79%
Early stopping!
[CV]  n_neurons=140, learning_rate=0.01, batch_size=500, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7f5daccfe378>, total=   7.0s
[CV] n_neurons=140, learning_rate=0.01, batch_size=500, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7f5daccfe378> 
0	Validation loss: 0.113734	Best loss: 0.113734	Accuracy: 96.36%
1	Validation loss: 0.072779	Best loss: 0.072779	Accuracy: 97.89%
2	Validation loss: 0.061158	Best loss: 0.061158	Accuracy: 97.93%
3	Validation loss: 0.057336	Best loss: 0.057336	Accuracy: 98.08%
4	Validation loss: 0.053134	Best loss: 0.053134	Accuracy: 98.32%
5	Validation loss: 0.051706	Best

5	Validation loss: 1.667118	Best loss: 1.662058	Accuracy: 18.73%
6	Validation loss: 1.852201	Best loss: 1.662058	Accuracy: 19.08%
7	Validation loss: 1.649380	Best loss: 1.649380	Accuracy: 19.27%
8	Validation loss: 1.899610	Best loss: 1.649380	Accuracy: 20.91%
9	Validation loss: 1.914810	Best loss: 1.649380	Accuracy: 19.27%
10	Validation loss: 1.686125	Best loss: 1.649380	Accuracy: 22.01%
11	Validation loss: 1.647474	Best loss: 1.647474	Accuracy: 20.91%
12	Validation loss: 1.853964	Best loss: 1.647474	Accuracy: 20.91%
13	Validation loss: 1.669570	Best loss: 1.647474	Accuracy: 22.01%
14	Validation loss: 1.703601	Best loss: 1.647474	Accuracy: 19.27%
15	Validation loss: 1.998326	Best loss: 1.647474	Accuracy: 19.08%
16	Validation loss: 1.656298	Best loss: 1.647474	Accuracy: 20.91%
17	Validation loss: 1.947680	Best loss: 1.647474	Accuracy: 20.91%
18	Validation loss: 2.143587	Best loss: 1.647474	Accuracy: 20.91%
19	Validation loss: 1.900222	Best loss: 1.647474	Accuracy: 20.91%
20	Validation l

17	Validation loss: 4.842723	Best loss: 0.150222	Accuracy: 88.94%
18	Validation loss: 5.778210	Best loss: 0.150222	Accuracy: 81.51%
19	Validation loss: 7.326347	Best loss: 0.150222	Accuracy: 83.82%
20	Validation loss: 4.470490	Best loss: 0.150222	Accuracy: 89.09%
21	Validation loss: 5.496844	Best loss: 0.150222	Accuracy: 84.95%
Early stopping!
[CV]  n_neurons=100, learning_rate=0.05, batch_size=100, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7f5db4e0a378>, total=  18.8s
[CV] n_neurons=100, learning_rate=0.05, batch_size=100, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7f5db4e0a378> 
0	Validation loss: 0.149954	Best loss: 0.149954	Accuracy: 96.17%
1	Validation loss: 2217.808350	Best loss: 0.149954	Accuracy: 18.57%
2	Validation loss: 11.900814	Best loss: 0.149954	Accuracy: 45.27%
3	Validation loss: 68.650154	Best loss: 0.149954	Accuracy: 58.76%
4	Validation loss: 57.475800	Best loss: 0.149954	Accuracy: 74.75%
5	Validation loss: 18.4758

22	Validation loss: 0.183005	Best loss: 0.119529	Accuracy: 96.72%
23	Validation loss: 0.224664	Best loss: 0.119529	Accuracy: 95.66%
Early stopping!
[CV]  n_neurons=100, learning_rate=0.02, batch_size=50, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7f5db4e0a378>, total=  34.3s
[CV] n_neurons=70, learning_rate=0.05, batch_size=10, activation=<function elu at 0x7f5dc2d7e268> 
0	Validation loss: 2.135650	Best loss: 2.135650	Accuracy: 19.27%
1	Validation loss: 1.842109	Best loss: 1.842109	Accuracy: 22.01%
2	Validation loss: 1.800001	Best loss: 1.800001	Accuracy: 18.73%
3	Validation loss: 1.873452	Best loss: 1.800001	Accuracy: 19.08%
4	Validation loss: 1.731745	Best loss: 1.731745	Accuracy: 19.27%
5	Validation loss: 1.666483	Best loss: 1.666483	Accuracy: 19.08%
6	Validation loss: 2.220948	Best loss: 1.666483	Accuracy: 19.27%
7	Validation loss: 2.015726	Best loss: 1.666483	Accuracy: 19.08%
8	Validation loss: 2.090927	Best loss: 1.666483	Accuracy: 19.08%
9	Validation 

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
rnd_search.best_params_

In [ ]:
y_pred = rnd_search.predict(under5_X_test)
accuracy_score(under5_y_test, y_pred)